In [1]:
# Prosessing libs
import pandas as pd
import os
import numpy as np
from fastparquet import ParquetFile
import glob

# PySpark
from pyspark.sql import SparkSession
from pyspark.sql import functions as psf

# Visualisation libs
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px


In [2]:
# change memory allocation for pyarrow
# os.environ['PYARROW_MEMORY_POOL_MAX_CHUNKSIZE'] = '3000000000'
# os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
# os.environ['PYSPARK_PYTHON'] = sys.executable

## Get the path of all files

In [3]:
os.getcwd()

'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\notebooks-contributions'

In [4]:
os.chdir("..")


In [5]:
home_dir = os.getcwd()
parquet_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data-test")
parquet_dir

# parquet_dir = r"E:\Rafael-data\type=way\state=latest"


'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test'

In [6]:
files = glob.glob(f'{parquet_dir}\\**\\*.parquet', recursive=True)
files


['c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test\\year=2022\\data_0.parquet',
 'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test\\year=2022\\data_1.parquet',
 'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test\\year=2022\\data_10.parquet',
 'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test\\year=2022\\data_11.parquet',
 'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test\\year=2022\\data_2.parquet',
 'c:\\Users\\milan\\One

In [7]:
# os.chdir("..")
# home_dir = os.getcwd()
# parquet_dir_node = os.path.join(home_dir, r"oshdb-contributions-parquet-data\type=node")
# parquet_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data")
# duckdb_dir = os.path.join(home_dir, r"notebooks-contributions\duckdb")
# test_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data\type=node\year=2023")


In [8]:
# files = glob.glob(f'{parquet_dir}\\**\\*.parquet', recursive=True)
# files


In [9]:
# files_location_lst = []

# for (path_dir, folder_names, file_names) in os.walk(parquet_dir_node):
#     for file_name in file_names:
#         if file_name.endswith(".parquet"):
#             file_location = os.path.join(path_dir, file_name)
#             files_location_lst.append(file_location)
# files_location_lst


## Load the all data with Apache PySpark

In [10]:
spark = SparkSession.builder.appName("ReadParquetFiles").getOrCreate()

df = spark.read.parquet(*files)

# df.show()


In [11]:
# df.count()

In [12]:
# spark.stop()

In [13]:
# df.describe().show()

In [14]:
# df.na.drop(how="any",subset=["building_area"]).show()

### Data transformation, Data types conversion

In [15]:
# Cast the 'changeset_timestamp' column to a LongType
df = df.withColumn('changeset_timestamp', psf.col(
    'changeset_timestamp').cast('long'))

# Use the 'changeset_timestamp' column to create a new 'datetime' column
df = df.withColumn('datetime', psf.from_unixtime(
    psf.col('changeset_timestamp')/1000, 'yyyy-MM-dd HH:mm:ss.SSSSSS'))

# Show the resulting DataFrame
# df.show()


In [16]:
# Use the 'changeset_timestamp' column to create a new 'datetime' column
df = df.withColumn('y_m', psf.from_unixtime(
    psf.col('changeset_timestamp')/1000, 'yyyy-MM'))

# Create a new column called 'year' containing the year from the 'datetime' column
df = df.withColumn('year', psf.year('datetime'))



# Show the resulting DataFrame
# df.show()


In [17]:
# Convert the 'tags' column to a string for filtering
df = df.withColumn("tags_str", df.tags.cast("string"))
df = df.withColumn("road_str", df.road.cast("string"))

In [18]:
str(df.columns)

"['contrib_id', 'valid_from', 'valid_to', 'osm_id', 'osm_version', 'changeset_id', 'changeset_timestamp', 'hashtags', 'editor', 'user_id', 'tags', 'tags_delta', 'primary_feature', 'building', 'road', 'is_area', 'area', 'area_delta', 'length', 'length_delta', 'contrib_type', 'country_iso_a3', 'country', 'centroid', 'geometry_type', 'geometry_valid', 'geometry', 'state', 'type', 'datetime', 'y_m', 'year', 'tags_str', 'road_str']"

In [19]:
# df.count()

In [20]:
# df.groupBy("road").count().show()

In [21]:
# df.groupBy("state").count().show()

In [22]:
df = df.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


In [23]:
# df.groupBy("building").count().show()

In [24]:
df.columns

['contrib_id',
 'valid_from',
 'valid_to',
 'osm_id',
 'osm_version',
 'changeset_id',
 'changeset_timestamp',
 'hashtags',
 'editor',
 'user_id',
 'tags',
 'tags_delta',
 'primary_feature',
 'building',
 'road',
 'is_area',
 'area',
 'area_delta',
 'length',
 'length_delta',
 'contrib_type',
 'country_iso_a3',
 'country',
 'centroid',
 'geometry_type',
 'geometry_valid',
 'geometry',
 'state',
 'type',
 'datetime',
 'y_m',
 'year',
 'tags_str',
 'road_str',
 'hashtags_un']

In [25]:
# df.count()

In [26]:
# df.show()

In [27]:
# Try to identify Vietnam abbretiation

#df_V = df.filter(psf.col("country").startswith("V"))
# df_V.groupBy("country").count().show()

# Create temporary view
#df_V.createOrReplaceTempView("TAB")
# spark.sql("SELECT DISTINCT * FROM TAB").show()
#spark.sql("SELECT DISTINCT country FROM TAB").show()


In [28]:
# Get the last edit
# max_date = df.selectExpr("max(datetime)").collect()
# max_date[0]


# Number of created buildings

### Data wrangling – buildings
- select all buildings (col building) with 1 value and NULL value
- use groupBy to count the last record of the users → the number of buildings

In [29]:
# df.groupBy("building").count().show()

In [30]:
df_blds_created = df.filter(psf.col('building').isin([1]) | psf.col('building').isNull()) # | psf.col('building').isNull()


In [31]:
# df_blds_created.show()

In [32]:
# df_blds = df.filter(df.building.contains(1) | df.building.contains(0) | df.building.isNull())


In [33]:
# df_blds.show()

In [34]:
# df_blds.groupBy("building").count().show()


In [35]:
df_blds_created = df_blds_created.groupBy("osm_id", "editor", "country", "tags_str", "hashtags", "year").agg(psf.last("datetime").alias("datetime"))

In [36]:
# df_blds.show()

In [37]:
# df_blds.show()

In [38]:
df_blds_created = df_blds_created.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


## Building filter

### Filter for numbers of  created buildings
- column tag_str contains 'building' to filter all buildings

In [39]:
df_all_blds_created = df_blds_created.filter(df_blds_created.tags_str.contains("building"))
# df_all_blds_created.show()

In [40]:
# World
df_all_blds_created_world = df_all_blds_created.groupBy("year").count().withColumnRenamed("count", "df_all_blds_created_world").toPandas()
df_all_blds_created_world


,year,df_all_blds_created_world
0,2023,1592
1,2022,52289008


In [41]:
# Selected countries
df_all_blds_created_country = df_all_blds_created.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_all_blds_created_country = df_all_blds_created_country.groupBy("year", "country").count().withColumnRenamed("count", "df_all_blds_created_country").toPandas()
df_all_blds_created_country

,year,country,df_all_blds_created_country
0,2022,USA,8779829
1,2022,NGA,2873253
2,2022,CZE,101841
3,2023,DEU,13
4,2023,USA,296
5,2022,VNM,70447
6,2023,NGA,144
7,2022,DEU,1968254


In [42]:
# # USA - USA
# df_all_blds_created_country = df_all_blds_created.groupBy("year", "country").count().withColumnRenamed("count", "df_all_blds_created_country_USA")
# df_all_blds_created_country_USA = df_all_blds_created_country.filter("country == 'USA' OR country == 'CZE'").toPandas()  # psf.col("country") == "USA" | psf.col("country") == "CZE"
# df_all_blds_created_country_USA


In [43]:
# # Nigeria - NGA
# df_all_blds_created_country = df_all_blds_created.groupBy("year", "country").count()
# df_all_blds_created_country_NGA = df_all_blds_created_country.filter(df_all_blds_created_country.country == "NGA").toPandas()
# df_all_blds_created_country_NGA

In [44]:
# # Vietnam - VNM
# df_all_blds_created_country = df_all_blds_created.groupBy("year", "country").count()
# df_all_blds_created_country_VNM = df_all_blds_created_country.filter(df_all_blds_created_country.country == "VNM").toPandas()
# df_all_blds_created_country_VNM

In [45]:
# # Germany - DEU
# df_all_blds_created_country = df_all_blds_created.groupBy("year", "country").count()
# df_all_blds_created_country_DEU = df_all_blds_created_country.filter(df_all_blds_created_country.country == "DEU").toPandas()
# df_all_blds_created_country_DEU

In [46]:
# # Czechia - CZE
# df_all_blds_created_country = df_all_blds_created.groupBy("year", "country").count()
# df_all_blds_created_country_CZE = df_all_blds_created_country.filter(df_all_blds_created_country.country == "CZE").toPandas()
# df_all_blds_created_country_CZE


## AI created buildings filter

### Filter 1
- column tag_str contains 'microsoft/BuildingFootprints' or 'esri/Google_Africa_Buildings'

In [47]:
df_ai_blds_created = df_all_blds_created.filter(
    df_all_blds_created.tags_str.contains("microsoft/BuildingFootprints")
    | df_all_blds_created.tags_str.contains("esri/Google_Africa_Buildings")
    )


In [48]:
# World
df_ai_blds_created_world = df_ai_blds_created.groupBy("year").count().withColumnRenamed("count", "df_ai_blds_created_world").toPandas()
df_ai_blds_created_world


,year,df_ai_blds_created_world
0,2023,138
1,2022,6293525


In [49]:
# Selected countries
df_ai_blds_created_country = df_ai_blds_created.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_ai_blds_created_country = df_ai_blds_created_country.groupBy("year", "country").count().withColumnRenamed("count", "df_ai_blds_created_country").toPandas()
df_all_blds_created_country

,year,country,df_all_blds_created_country
0,2022,USA,8779829
1,2022,NGA,2873253
2,2022,CZE,101841
3,2023,DEU,13
4,2023,USA,296
5,2022,VNM,70447
6,2023,NGA,144
7,2022,DEU,1968254


In [50]:
# # USA - USA
# df_ai_blds_created_country = df_ai_blds_created.groupBy("year", "country").count()
# df_ai_blds_created_country_USA = df_ai_blds_created_country.filter(df_ai_blds_created_country.country == "USA").toPandas()
# df_ai_blds_created_country_USA

In [51]:
# # Nigeria - NGA
# df_ai_blds_created_country = df_ai_blds_created.groupBy("year", "country").count()
# df_ai_blds_created_country_NGA = df_ai_blds_created_country.filter(df_ai_blds_created_country.country == "NGA").toPandas()
# df_ai_blds_created_country_NGA

In [52]:
# # Vietnam - VNM
# df_ai_blds_created_country = df_ai_blds_created.groupBy("year", "country").count()
# df_ai_blds_created_country_VNM = df_ai_blds_created_country.filter(df_ai_blds_created_country.country == "VNM").toPandas()
# df_ai_blds_created_country_VNM


In [53]:
# # Germany - DEU
# df_ai_blds_created_country = df_ai_blds_created.groupBy("year", "country").count()
# df_ai_blds_created_country_DEU = df_ai_blds_created_country.filter(df_ai_blds_created_country.country == "DEU").toPandas()
# df_ai_blds_created_country_DEU

In [54]:
# # Czechia - CZE
# df_ai_blds_created_country = df_ai_blds_created.groupBy("year", "country").count()
# df_ai_blds_created_country_CZE = df_ai_blds_created_country.filter(df_ai_blds_created_country.country == "CZE").toPandas()
# df_ai_blds_created_country_CZE

# Number of edited buildings

In [55]:
df_blds_edited = df.filter(psf.col('building').isin([0]) | psf.col('building').isNull()) # | psf.col('building').isNull()


In [56]:
# df_blds = df.filter(df.building.contains(1) | df.building.contains(0) | df.building.isNull())


In [57]:
# df_blds.show()

In [58]:
# df_blds.groupBy("building").count().show()


In [59]:
df_blds_edited = df_blds_edited.groupBy("osm_id", "editor", "country", "tags_str", "hashtags", "year").agg(psf.last("datetime").alias("datetime"))


In [60]:
# df_blds.show()

In [61]:
# df_blds.show()

In [62]:
df_blds_edited = df_blds_edited.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


## Building filter

### Filter for numbers of  edited buildings
- column tag_str contains 'building' to filter all buildings

In [63]:
df_all_blds_edited = df_blds_edited.filter(df_blds_edited.tags_str.contains("building"))


In [64]:
# World
df_all_blds_edited_world = df_all_blds_edited.groupBy("year").count().withColumnRenamed("count", "df_all_blds_edited_world").toPandas()
df_all_blds_edited_world

,year,df_all_blds_edited_world
0,2023,515
1,2022,28913691


In [65]:
# Selected countries
df_all_blds_edited_country = df_all_blds_edited.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_all_blds_edited_country = df_all_blds_edited_country.groupBy("year", "country").count().withColumnRenamed("count", "df_all_blds_edited_country").toPandas()
df_all_blds_edited_country

,year,country,df_all_blds_edited_country
0,2022,USA,2781094
1,2022,NGA,974370
2,2022,CZE,258883
3,2023,DEU,114
4,2023,USA,67
5,2022,VNM,17566
6,2022,DEU,3280971


In [66]:
# # USA - USA
# df_all_blds_edited_country = df_all_blds_edited.groupBy("year", "country").count()
# df_all_blds_edited_country_USA = df_all_blds_edited_country.filter(df_all_blds_edited_country.country == "USA").toPandas()
# df_all_blds_edited_country_USA

In [67]:
# # Nigeria - NGA
# df_all_blds_edited_country = df_all_blds_edited.groupBy("year", "country").count()
# df_all_blds_edited_country_NGA = df_all_blds_edited_country.filter(df_all_blds_edited_country.country == "NGA").toPandas()
# df_all_blds_edited_country_NGA

In [68]:
# # Vietnam - VNM
# df_all_blds_edited_country = df_all_blds_edited.groupBy("year", "country").count()
# df_all_blds_edited_country_VNM = df_all_blds_edited_country.filter(df_all_blds_edited_country.country == "VNM").toPandas()
# df_all_blds_edited_country_VNM


In [69]:
# # Germany - DEU
# df_all_blds_edited_country = df_all_blds_edited.groupBy("year", "country").count()
# df_all_blds_edited_country_DEU = df_all_blds_edited_country.filter(df_all_blds_edited_country.country == "DEU").toPandas()
# df_all_blds_edited_country_DEU

In [70]:
# # Czechia - CZE
# df_all_blds_edited_country = df_all_blds_edited.groupBy("year", "country").count()
# df_all_blds_edited_country_CZE = df_all_blds_edited_country.filter(df_all_blds_edited_country.country == "CZE").toPandas()
# df_all_blds_edited_country_CZE

## AI edited buildings filter

### Filter 1
- column tag_str contains 'microsoft/BuildingFootprints' or 'esri/Google_Africa_Buildings'

In [71]:
df_ai_blds_edited = df_all_blds_edited.filter(
    df_all_blds_edited.tags_str.contains("microsoft/BuildingFootprints")
    | df_all_blds_edited.tags_str.contains("esri/Google_Africa_Buildings")
    )


In [72]:
# World
df_ai_blds_edited_world = df_ai_blds_edited.groupBy("year").count().withColumnRenamed("count", "df_ai_blds_edited_world").toPandas()
df_ai_blds_edited_world


,year,df_ai_blds_edited_world
0,2023,6
1,2022,932986


In [73]:
# Selected countries
df_ai_blds_edited_country = df_ai_blds_edited.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_ai_blds_edited_country = df_ai_blds_edited_country.groupBy("year", "country").count().withColumnRenamed("count", "df_ai_blds_edited_country").toPandas()
df_ai_blds_edited_country

,year,country,df_ai_blds_edited_country
0,2022,USA,555929
1,2022,NGA,195584
2,2023,USA,4
3,2022,DEU,450
4,2022,CZE,1


In [74]:
# # USA - USA
# df_ai_blds_edited_country = df_ai_blds_edited.groupBy("year", "country").count()
# df_ai_blds_edited_country_USA = df_ai_blds_edited_country.filter(df_ai_blds_edited_country.country == "USA").toPandas()
# df_ai_blds_edited_country_USA


In [75]:
# # Nigeria - NGA
# df_ai_blds_edited_country = df_ai_blds_edited.groupBy("year", "country").count()
# df_ai_blds_edited_country_NGA = df_ai_blds_edited_country.filter(df_ai_blds_edited_country.country == "NGA").toPandas()
# df_ai_blds_edited_country_NGA

In [76]:
# # Vietnam - VNM
# df_ai_blds_edited_country = df_ai_blds_edited.groupBy("year", "country").count()
# df_ai_blds_edited_country_VNM = df_ai_blds_edited_country.filter(df_ai_blds_edited_country.country == "VNM").toPandas()
# df_ai_blds_edited_country_VNM

In [77]:
# # Germany - DEU
# df_ai_blds_edited_country = df_ai_blds_edited.groupBy("year", "country").count()
# df_ai_blds_edited_country_DEU = df_ai_blds_edited_country.filter(df_ai_blds_edited_country.country == "DEU").toPandas()
# df_ai_blds_edited_country_DEU

In [78]:
# # Czechia - CZE
# df_ai_blds_edited_country = df_ai_blds_edited.groupBy("year", "country").count()
# df_ai_blds_edited_country_CZE = df_ai_blds_edited_country.filter(df_ai_blds_edited_country.country == "CZE").toPandas()
# df_ai_blds_edited_country_CZE

# Length of created roads

### Data wrangling – roads
- select distinct osm_id
- remove all -1
- then use groupby (osm_id) and function last (time column)

In [79]:
df_roads_created = df.filter(psf.col('road').isin([1]) | psf.col('road').isNull()) # | psf.col('road').isNull()

In [80]:
# df_roads = df.filter(~psf.col('road').isin([-1, 0]))

In [81]:
df_roads_created = df_roads_created.groupBy("osm_id", "editor", "length", "country", "tags_str", "hashtags", "year", "road").agg(psf.last("datetime").alias("datetime"))


In [82]:
# df_roads.show()

In [83]:
df_roads_created = df_roads_created.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


In [84]:
# df_roads.groupBy("hashtags_un").count().show()


In [85]:
# df_roads.show()

## Road filter

### Filter for the length of all roads
- column tag_str contains motorway, trunk, motorway_link, trunk_link, primary, primary_link, secondary, secondary_link, tertiary, tertiary_link, unclassified, residential

In [86]:
df_main_roads_created = df_roads_created.filter(
    df_roads_created.tags_str.contains("motorway")\
    | df_roads_created.tags_str.contains("trunk")\
    | df_roads_created.tags_str.contains("motorway_link")\
    | df_roads_created.tags_str.contains("trunk_link")\
    | df_roads_created.tags_str.contains("primary")\
    | df_roads_created.tags_str.contains("primary_link")\
    | df_roads_created.tags_str.contains("secondary")\
    | df_roads_created.tags_str.contains("secondary_link")\
    | df_roads_created.tags_str.contains("tertiary")\
    | df_roads_created.tags_str.contains("tertiary_link")\
    | df_roads_created.tags_str.contains("unclassified")\
    | df_roads_created.tags_str.contains("residential")\
    )


#### All roads

In [87]:
# World – all roads
df_all_roads_created_world = df_roads_created.groupBy("year").sum().toPandas()
df_all_roads_created_world


,year,sum(osm_id),sum(length),sum(year),sum(road)
0,2023,4052978511385,1684453,9184420,209
1,2022,112430116003053421,20090717326,257533392828,4660229


In [88]:
# Selected countries – all roads
df_all_roads_created_country = df_roads_created.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_all_roads_created_country = df_all_roads_created_country.groupBy("year", "country").sum().toPandas()
df_all_roads_created_country

,year,country,sum(osm_id),sum(length),sum(year),sum(road)
0,2022,USA,18162075950283805,2737851895,39315774066,411999.0
1,2022,NGA,4122035800093102,76092852,7984107618,61021.0
2,2022,CZE,438570221239721,147976224,1613683386,14172.0
3,2023,DEU,159659026223,58728,746487,6.0
4,2023,USA,1107948439008,268761,2338588,26.0
5,2022,VNM,170752223858560,52606864,350066838,25116.0
6,2023,NGA,181754598639,16143,329749,16.0
7,2022,DEU,5806913758696467,789885217,18769847886,66478.0
8,2023,CZE,5518540754,526,14161,NaN


#### Main roads

In [89]:
# World – main roads
df_main_roads_created_world = df_main_roads_created.groupBy("year").sum().toPandas()
df_main_roads_created_world


,year,sum(osm_id),sum(length),sum(year),sum(road)
0,2023,305049589226,92700,720188,209
1,2022,8545669238698607,1585870001,18457208268,4660229


In [90]:
# Selected countries – main roads
df_main_roads_created_country = df_main_roads_created.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_main_roads_created_country = df_main_roads_created_country.groupBy("year", "country").sum().toPandas()
df_main_roads_created_country

,year,country,sum(osm_id),sum(length),sum(year),sum(road)
0,2022,USA,612863536403005,110489102,1252226622,411999.0
1,2022,NGA,132497126249363,16822783,290632170,61021.0
2,2022,CZE,41774418878223,2650393,213238098,14172.0
3,2023,DEU,15728191896,757,103173,6.0
4,2023,USA,62659560899,8000,123403,26.0
5,2022,VNM,28362641214638,6903442,54773958,25116.0
6,2023,NGA,18543158313,10777,34391,16.0
7,2022,DEU,199411478811635,10029466,669939150,66478.0
8,2023,CZE,5438511905,347,12138,NaN


In [91]:
# # USA - USA
# df_all_roads_created_country = df_all_roads_created.groupBy("year", "country").count()
# df_all_roads_created_country.filter(df_all_roads_created_country.country == "USA").show()

In [92]:
# # Nigeria - NGA
# df_all_roads_created_country = df_all_roads_created.groupBy("year", "country").count()
# df_all_roads_created_country.filter(df_all_roads_created_country.country == "NGA").toPandas()

In [93]:
# # Vietnam - VNM
# df_all_roads_created_country = df_all_roads_created.groupBy("year", "country").count()
# df_all_roads_created_country.filter(df_all_roads_created_country.country == "VNM").show()

In [94]:
# # Germany - DEU
# df_all_roads_created_country = df_all_roads_created.groupBy("year", "country").count()
# df_all_roads_created_country.filter(df_all_roads_created_country.country == "DEU").toPandas()

In [95]:
# # Czechia - CZE
# df_all_roads_created_country = df_all_roads_created.groupBy("year", "country").count()
# df_all_roads_created_country.filter(df_all_roads_created_country.country == "CZE").show()

## AI road filter

### Filter 1
- column tags contains “#mapwithai” or “#nsroadimport” or "#MapWithAI"

In [96]:
# df_roads_f1_created = df_roads_created.filter(\
#     df_roads_created.tags_str.contains("mapwithai")\
#     | df_roads_created.tags_str.contains("MapWithAI")\
#     | df_roads_created.tags_str.contains("nsroadimport")\
#     )


In [97]:
# df_roads_f1_created.groupBy("year").sum().show()


### Filter 2
- column tags_str contains (#nsroadimport" or "mapwithai" or “MapWithAI") and "highway"
OR
- column hashtags_un contains (#nsroadimport" or "mapwithai") and it is highway (from column tags_str ???)

In [98]:
# df_roads_f2_created = df_roads_created.filter(\
#     (
#         (df_roads_created.tags_str.contains("mapwithai")\
#         | df_roads_created.tags_str.contains("MapWithAI")\
#         | df_roads_created.tags_str.contains("nsroadimport"))\
#         & (df_roads_created.tags_str.contains("highway")))\
#         |\
#         (df_roads_created.hashtags_un.contains("mapwithai")\
#         | df_roads_created.hashtags_un.contains("nsroadimport")\
#         & (df_roads_created.tags_str.contains("highway")))\
#     )


In [99]:
# df_roads_f2_created.groupBy("year").sum().show()


### Filter 3
3_1 – column editor contains RapiD

3_2 – column editor contains RapiD or column tags_str/hashtags_un contains mapwithai

3_3 – column editor contains RapiD and column tags_str/hashtags_un contains mapwithai

#### F3_1

In [100]:
# df_roads_f3_1_created = df_roads_created.filter(df_roads_created.editor.contains("RapiD"))


In [101]:
# df_roads_f3_1_created.groupBy("year").sum().show()


#### F3_2

In [102]:
df_all_ai_roads_f3_2_created = df_roads_created.filter(
    df_roads_created.editor.contains("RapiD")\
    | df_roads_created.tags_str.contains("mapwithai")\
    | df_roads_created.tags_str.contains("MapWithAI")\
    | df_roads_created.tags_str.contains("nsroadimport")\
    | df_roads_created.hashtags_un.contains("#mapwithai")\
    | df_roads_created.hashtags_un.contains("#nsroadimport")\
    | df_roads_created.hashtags_un.contains("#MapWithAI")\
)


In [103]:
df_main_ai_roads_f3_2_created = df_main_roads_created.filter(
    df_main_roads_created.editor.contains("RapiD")
    | df_main_roads_created.tags_str.contains("mapwithai")
    | df_main_roads_created.tags_str.contains("MapWithAI")
    | df_main_roads_created.tags_str.contains("nsroadimport")
    | df_main_roads_created.hashtags_un.contains("#mapwithai")
    | df_main_roads_created.hashtags_un.contains("#nsroadimport")
    | df_main_roads_created.hashtags_un.contains("#MapWithAI")
)


#### All AI roads 

In [104]:
# World
df_all_ai_roads_created_world = df_all_ai_roads_f3_2_created.groupBy("year").sum().toPandas()
df_all_ai_roads_created_world


,year,sum(osm_id),sum(length),sum(year),sum(road)
0,2023,230135266671,5355,424830,4
1,2022,7352139887482507,674552968,14380269888,1115263


In [105]:
# Selected countries
df_all_ai_roads_created_country = df_all_ai_roads_f3_2_created.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_all_ai_roads_created_country = df_all_ai_roads_created_country.groupBy("year", "country").sum().toPandas()
df_all_ai_roads_created_country

,year,country,sum(osm_id),sum(length),sum(year),sum(road)
0,2022,USA,3422392598178235,55804445,6583947432,8506
1,2022,NGA,367978405705376,1627735,703817760,3161
2,2022,CZE,127503966371,17638,266904,8
3,2023,USA,230135266671,5355,424830,4
4,2022,VNM,329441550790,179822,608622,108
5,2022,DEU,5416763449045,161554,12051120,14


#### Main AI roads

In [106]:
# World
df_main_ai_roads_created_world = df_main_ai_roads_f3_2_created.groupBy("year").sum().toPandas()
df_main_ai_roads_created_world


,year,sum(osm_id),sum(length),sum(year),sum(road)
0,2022,1195229411870664,237371841,2305573368,1115263
1,2023,4504479606,1284,8092,4


In [107]:
# Selected countries
df_main_ai_roads_created_country = df_main_ai_roads_f3_2_created.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_main_ai_roads_created_country = df_main_ai_roads_created_country.groupBy("year", "country").sum().toPandas()
df_main_ai_roads_created_country

,year,country,sum(osm_id),sum(length),sum(year),sum(road)
0,2022,USA,12757623127701,2499279,29337198,8506
1,2022,NGA,3563479791279,960916,6795942,3161
2,2022,CZE,9114090399,1015,18198,8
3,2023,USA,4504479606,1284,8092,4
4,2022,VNM,121340191548,63061,218376,108
5,2022,DEU,190576058708,848,414510,14


In [108]:
# # USA - USA
# df_roads_f3_2_created_country = df_roads_f3_2_created.groupBy("year", "country").count()
# df_roads_f3_2_created_country.filter(df_roads_f3_2_created_country.country == "USA").show()

In [109]:
# # Nigeria - NGA
# df_roads_f3_2_created_country = df_roads_f3_2_created.groupBy("year", "country").count()
# df_roads_f3_2_created_country.filter(df_roads_f3_2_created_country.country == "NGA").toPandas()

In [110]:
# # Vietnam - VN
# df_roads_f3_2_created_country = df_roads_f3_2_created.groupBy("year", "country").count()
# df_roads_f3_2_created_country.filter(df_roads_f3_2_created_country.country == "VNM").show()

In [111]:
# # Germany - DEU
# df_roads_f3_2_created_country = df_roads_f3_2_created.groupBy("year", "country").count()
# df_roads_f3_2_created_country.filter(df_roads_f3_2_created_country.country == "DEU").toPandas()

In [112]:
# # Czechia - CZE
# df_roads_f3_2_created_country = df_roads_f3_2_created.groupBy("year", "country").count()
# df_roads_f3_2_created_country.filter(df_roads_f3_2_created_country.country == "CZE").show()

#### F3_3

In [113]:
# df_roads_f3_3_created = df_roads_created.filter(\
#     (df_roads_created.editor.contains("RapiD")\
#      & (df_roads_created.tags_str.contains("mapwithai")\
#         | df_roads_created.hashtags_un.contains("mapwithai")\
#         | df_roads_created.tags_str.contains("MapWithAI")\
#         | df_roads_created.hashtags_un.contains("MapWithAI")))\
#     )


In [114]:
# df_roads_f3_3_created.groupBy("year").sum().show()


# Length of edited roads

In [115]:
df_roads_edited = df.filter(psf.col('road').isin([0]) | psf.col('road').isNull()) #  | psf.col('road').isNull()

In [116]:
# df_roads = df.filter(~psf.col('road').isin([-1, 0]))

In [117]:
df_roads_edited = df_roads_edited.groupBy("osm_id", "editor", "length", "country", "tags_str", "hashtags", "year", "road").agg(psf.last("datetime").alias("datetime"))


In [118]:
# df_roads.show()

In [119]:
df_roads_edited = df_roads_edited.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


In [120]:
# df_roads.groupBy("hashtags_un").count().show()


In [121]:
# df_roads.show()

## Edited roads filter

### Filter for the length of all edited roads
- column tag_str contains motorway, trunk, motorway_link, trunk_link, primary, primary_link, secondary, secondary_link, tertiary, tertiary_link, unclassified, residential

In [122]:
df_main_roads_edited = df_roads_edited.filter(
    df_roads_edited.tags_str.contains("motorway")\
    | df_roads_edited.tags_str.contains("trunk")\
    | df_roads_edited.tags_str.contains("motorway_link")\
    | df_roads_edited.tags_str.contains("trunk_link")\
    | df_roads_edited.tags_str.contains("primary")\
    | df_roads_edited.tags_str.contains("primary_link")\
    | df_roads_edited.tags_str.contains("secondary")\
    | df_roads_edited.tags_str.contains("secondary_link")\
    | df_roads_edited.tags_str.contains("tertiary")\
    | df_roads_edited.tags_str.contains("tertiary_link")\
    | df_roads_edited.tags_str.contains("unclassified")\
    | df_roads_edited.tags_str.contains("residential")\
    )


#### All roads

In [123]:
# World – all roads
df_all_roads_edited_world = df_roads_edited.groupBy("year").sum().toPandas()
df_all_roads_edited_world


,year,sum(osm_id),sum(length),sum(year),sum(road)
0,2023,4518769711821,2496515,12115747,0
1,2022,114295611440936595,28716825685,280682049540,0


In [124]:
# Selected countries – all roads
df_all_roads_edited_country = df_roads_edited.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_all_roads_edited_country = df_all_roads_edited_country.groupBy("year", "country").sum().toPandas()
df_all_roads_edited_country

,year,country,sum(osm_id),sum(length),sum(year),sum(road)
0,2022,USA,18385360283023343,4213787767,42908718438,0.0
1,2022,NGA,4121824319706726,150751101,8064840012,0.0
2,2022,CZE,453634520157374,178607925,1786875774,0.0
3,2023,DEU,243446563119,92679,1191547,0.0
4,2023,USA,1180638761896,538641,2998086,0.0
5,2022,VNM,205330138759545,103359830,477982602,0.0
6,2023,NGA,165164785249,9411,303450,0.0
7,2022,DEU,5979916376125585,997512191,20542990236,0.0
8,2023,CZE,5518540754,526,14161,NaN


#### Main roads

In [125]:
# World – main roads
df_main_roads_edited_world = df_main_roads_edited.groupBy("year").sum().toPandas()
df_main_roads_edited_world


,year,sum(osm_id),sum(length),sum(year),sum(road)
0,2023,770840789662,904762,3651515,0
1,2022,10411164676581781,10211978360,41605864980,0


In [126]:
# Selected countries – main roads
df_main_roads_edited_country = df_main_roads_edited.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_main_roads_edited_country = df_main_roads_edited_country.groupBy("year", "country").sum().toPandas()
df_main_roads_edited_country

,year,country,sum(osm_id),sum(length),sum(year),sum(road)
0,2022,USA,836147869142543,1586424974,4845170994,0.0
1,2022,NGA,132285645862987,91481032,371364564,0.0
2,2022,CZE,56838717795876,33282094,386430486,0.0
3,2023,DEU,99515728792,34708,548233,0.0
4,2023,USA,135349883787,277880,782901,0.0
5,2022,VNM,62940556115623,57656408,182689722,0.0
6,2023,NGA,1953344923,4045,8092,0.0
7,2022,DEU,372414096240753,217656440,2443081500,0.0
8,2023,CZE,5438511905,347,12138,NaN


In [127]:
# # World
# df_all_roads_edited.groupBy("year").sum().toPandas()

In [128]:
# # USA - USA
# df_all_roads_edited_country = df_all_roads_edited.groupBy("year", "country").count()
# df_all_roads_edited_country.filter(df_all_roads_edited_country.country == "USA").show()

In [129]:
# # Nigeria - NGA
# df_all_roads_edited_country = df_all_roads_edited.groupBy("year", "country").count()
# df_all_roads_edited_country.filter(df_all_roads_edited_country.country == "NGA").toPandas()

In [130]:
# # Vietnam - VN
# df_all_roads_edited_country = df_all_roads_edited.groupBy("year", "country").count()
# df_all_roads_edited_country.filter(df_all_roads_edited_country.country == "VNM").show()

In [131]:
# # Germany - DEU
# df_all_roads_edited_country = df_all_roads_edited.groupBy("year", "country").count()
# df_all_roads_edited_country.filter(df_all_roads_edited_country.country == "DEU").toPandas()

In [132]:
# # Czechia - CZE
# df_all_roads_edited_country = df_all_roads_edited.groupBy("year", "country").count()
# df_all_roads_edited_country.filter(df_all_roads_edited_country.country == "CZE").show()

## AI road filter

### Filter 1
- column tags contains “#mapwithai” or “#nsroadimport” or "#MapWithAI"

In [133]:
# df_roads_f1_edited = df_roads_edited.filter(\
#     df_roads_edited.tags_str.contains("mapwithai")\
#     | df_roads_edited.tags_str.contains("MapWithAI")\
#     | df_roads_edited.tags_str.contains("nsroadimport")\
#     )


In [134]:
# df_roads_f1_edited.groupBy("year").sum().show()


### Filter 2
- column tags_str contains (#nsroadimport" or "mapwithai" or “MapWithAI") and "highway"
OR
- column hashtags_un contains (#nsroadimport" or "mapwithai") and it is highway (from column tags_str ???)

In [135]:
# df_roads_f2_edited = df_roads_edited.filter(\
#     (\
#         (df_roads_edited.tags_str.contains("mapwithai")\
#          | df_roads_edited.tags_str.contains("MapWithAI")\
#          | df_roads_edited.tags_str.contains("nsroadimport"))\
#         & (df_roads_edited.tags_str.contains("highway")))\
#       |\
#     (df_roads_edited.hashtags_un.contains("mapwithai")\
#          | df_roads_edited.hashtags_un.contains("nsroadimport")\
#          & (df_roads_edited.tags_str.contains("highway"))))


In [136]:
# df_roads_f2_edited.groupBy("year").sum().show()


### Filter 3
3_1 – column editor contains RapiD

3_2 – column editor contains RapiD or column tags_str/hashtags_un contains mapwithai

3_3 – column editor contains RapiD and column tags_str/hashtags_un contains mapwithai

#### F3_1

In [137]:
# df_roads_f3_1_edited = df_roads_edited.filter(df_roads_edited.editor.contains("RapiD"))


In [138]:
# df_roads_f3_1_edited.groupBy("year").sum().show()


#### F3_2

In [139]:
df_all_ai_roads_f3_2_edited = df_roads_edited.filter(
    df_roads_edited.editor.contains("RapiD")\
    | df_roads_edited.tags_str.contains("mapwithai")\
    | df_roads_edited.tags_str.contains("MapWithAI")\
    | df_roads_edited.tags_str.contains("nsroadimport")\
    | df_roads_edited.hashtags_un.contains("#mapwithai")\
    | df_roads_edited.hashtags_un.contains("#nsroadimport")\
    | df_roads_edited.hashtags_un.contains("#MapWithAI")\
)


In [140]:
df_main_ai_roads_f3_2_edited = df_main_roads_edited.filter(
    df_roads_edited.editor.contains("RapiD")
    | df_roads_edited.tags_str.contains("mapwithai")
    | df_roads_edited.tags_str.contains("MapWithAI")
    | df_roads_edited.tags_str.contains("nsroadimport")
    | df_roads_edited.hashtags_un.contains("#mapwithai")
    | df_roads_edited.hashtags_un.contains("#nsroadimport")
    | df_roads_edited.hashtags_un.contains("#MapWithAI")
)


#### All AI roads 

In [141]:
# World
df_all_ai_roads_edited_world = df_all_ai_roads_f3_2_edited.groupBy("year").sum().toPandas()
df_all_ai_roads_edited_world


,year,sum(osm_id),sum(length),sum(year),sum(road)
0,2023,231160203391,12624,443037,0
1,2022,6857344568891178,1019119588,14093985018,0


In [142]:
# Selected countries
df_all_ai_roads_edited_country = df_all_ai_roads_f3_2_edited.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
df_all_ai_roads_edited_country = df_all_ai_roads_edited_country.groupBy("year", "country").sum().toPandas()
df_all_ai_roads_edited_country

,year,country,sum(osm_id),sum(length),sum(year),sum(road)
0,2022,USA,3440251410162982,93726838,6742193196,0
1,2022,NGA,365908383148808,2937656,702319458,0
2,2022,CZE,120265157555,27154,266904,0
3,2023,USA,231160203391,12624,443037,0
4,2022,VNM,262980817010,240862,545940,0
5,2022,DEU,5433421142340,180996,12249276,0


#### Main AI roads

In [143]:
# # World
# df_main_ai_roads_edited_world = df_main_ai_roads_f3_2_edited.groupBy("year").sum().toPandas()
# df_main_ai_roads_edited_world


In [144]:
# # Selected countries
# df_main_ai_roads_edited_country = df_main_ai_roads_f3_2_edited.filter("country == 'USA' OR country == 'NGA' OR country == 'VNM' OR country == 'DEU' OR country == 'CZE'")
# df_main_ai_roads_edited_country = df_main_ai_roads_edited_country.groupBy("year", "country").count().withColumnRenamed("count", "df_main_ai_roads_created_country").toPandas()
# df_main_ai_roads_edited_country

In [145]:
# # World
# df_roads_f3_2_edited.groupBy("year").sum().toPandas()


In [146]:
# # USA - USA
# df_roads_f3_2_edited_country = df_roads_f3_2_edited.groupBy("year", "country").count()
# df_roads_f3_2_edited_country.filter(df_roads_f3_2_edited_country.country == "USA").show()

In [147]:
# # Nigeria - NGA
# df_roads_f3_2_edited_country = df_roads_f3_2_edited.groupBy("year", "country").count()
# df_roads_f3_2_edited_country.filter(df_roads_f3_2_edited_country.country == "NGA").toPandas()

In [148]:
# # Vietnam - VN
# df_roads_f3_2_edited_country = df_roads_f3_2_edited.groupBy("year", "country").count()
# df_roads_f3_2_edited_country.filter(df_roads_f3_2_edited_country.country == "VNM").show()

In [149]:
# # Germany - DEU
# df_roads_f3_2_edited_country = df_roads_f3_2_edited.groupBy("year", "country").count()
# df_roads_f3_2_edited_country.filter(df_roads_f3_2_edited_country.country == "DEU").toPandas()

In [150]:
# # Czechia - CZE
# df_roads_f3_2_edited_country = df_roads_f3_2_edited.groupBy("year", "country").count()
# df_roads_f3_2_edited_country.filter(df_roads_f3_2_edited_country.country == "CZE").show()

#### F3_3

In [151]:
# df_roads_f3_3_edited = df_roads_edited.filter(\
#     (df_roads_edited.editor.contains("RapiD")\
#      & (df_roads_edited.tags_str.contains("mapwithai")\
#         | df_roads_edited.hashtags_un.contains("mapwithai")\
#         | df_roads_edited.tags_str.contains("MapWithAI")\
#         | df_roads_edited.hashtags_un.contains("MapWithAI")))\
#     )


In [152]:
# df_roads_f3_3_edited.groupBy("year").sum().show()


## Plot the numbers

In [153]:
class BarPlotter:
    def __init__(self, *dfs, x_column, y_column):
        self.dfs = list(dfs)
        self.x_column = x_column
        self.y_column = y_column
        self.N = len(self.dfs[0])
        self.width = 0.2

    def create_bar_plot(self, title=None, xlabel="X-axis", ylabel="Y-axis"):
        if title is None:
            title = [
                f'DataFrame {self.dfs[i]["country"][0]}' for i in range(len(self.dfs))]
        ind = np.arange(len(self.dfs[0]))
        plt.figure(figsize=(10, 6))

        for i, df in enumerate(self.dfs):
            plt.bar(ind + i * self.width,
                    df[self.y_column], self.width, label=title[i])

        # plt.bar(ind,
        #         self.df1[self.y_column], self.width,  label=self.df1["country"][0])
        # plt.bar(ind, self.df2[self.y_column], self.width, df[self.y_column], self.width, label=titles[i])
        #         label=self.df2["country"][0])

        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        # plt.title(title)
        plt.legend()
        # plt.xticks(rotation=45)
        plt.xticks(ind + self.width * (self.N - 1) /
                   2, self.dfs[0][self.x_column])
        # plt.yticks(np.arange(len(self.df1[self.y_column])))
        plt.tight_layout()
        plt.show()

In [154]:
pl1 = BarPlotter(df_all_ai_roads_created_country.loc[:, ["country" == "USA"]], df_all_ai_roads_edited_country.loc[:, ["country" == "USA"]],
                 , x_column="year", y_column="count")
pl1.create_bar_plot(title=None, xlabel="Year", ylabel="Count")

NameError: name 'df_all_blds_created_country_USA' is not defined

# This chapter is useless so far

## Convert to Pandas DataFrame - AI buildings, AI roads (this chapter is useless so far)

In [ ]:
# # Filter AI blds
# df_blds = df.filter(df.tags_str.contains("microsoft/BuildingFootprints") | df.tags_str.contains("esri/Google_Africa_Buildings"))\
# .groupby("country", "editor", "year", "tags_str")\
# .agg(psf.sum("user_id"))\
# .withColumnRenamed("sum(user_id)", "count_u_id")\
# .sort(psf.col("count_u_id").desc())\
# .toPandas()


In [ ]:
# df_blds

,country,editor,year,tags_str,count_u_id
0,TUR,RapiD 1.1.9-tm.1,2023,"{building -> yes, source -> microsoft/Building...",1862607831910
1,IND,RapiD 1.1.9,2023,"{building -> yes, source -> microsoft/Building...",697575610154
2,IDN,JOSM/1.5 (18583 en),2023,"{building -> yes, source -> microsoft/Building...",167757823842
3,USA,RapiD 1.1.9,2023,"{building -> yes, source -> microsoft/Building...",116856923771
4,SYR,RapiD 1.1.9-tm.1,2023,"{building -> yes, source -> microsoft/Building...",97647827025
...,...,...,...,...,...
24984,GBR,StreetComplete 50.2,2023,"{building -> house, source -> microsoft/Buildi...",7329
24985,GBR,StreetComplete 50.2,2023,"{building -> detached, source -> microsoft/Bui...",7329
24986,GBR,StreetComplete 50.2,2023,"{building -> apartments, source -> microsoft/B...",7329
24987,GBR,RapiD 1.1.9,2023,"{building -> yes, source -> microsoft/Building...",7329


In [ ]:
# df_blds.notna().all(axis=1)


0        True
1        True
2        True
3        True
4        True
         ... 
24984    True
24985    True
24986    True
24987    True
24988    True
Length: 24989, dtype: bool

In [ ]:
# & (psf.col("road") == 1)


In [ ]:
# # Filter AI roads
# df_roads = df_roads.filter(((df_roads.tags_str.contains("mapwithai") | df_roads.tags_str.contains("MapWithAI") | df_roads.tags_str.contains("#nsroadimport")) & (df_roads.tags_str.contains("highway")))
#                            | (df_roads.hashtags_un.contains("#mapwithai") | df_roads.hashtags_un.contains("#nsroadimport")))\
#     .groupby("country", "editor", "year", "tags_str", "hashtags_un", "length")\
#     .agg(psf.sum("country"))\
#     .withColumnRenamed("sum(country)", "count_country")\
#     .sort(psf.col("count_country").desc())\
#     .toPandas()


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\milan\general_py_venv\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\milan\general_py_venv\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\milan\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# df_roads


,country,editor,year,tags_str,hashtags_un,length,count_osm_id
0,TZA,JOSM/1.5 (18646 en),2023,{},"#Tanzania,#mapwithai",0,3883303105856
1,TZA,JOSM/1.5 (18583 en),2023,{},"#Tanzania,#mapwithai",0,2204724899226
2,IND,RapiD 1.1.9-tm.1,2023,{},"#India,#mapwithai",0,1786006015589
3,IND,JOSM/1.5 (18427 en),2023,{},"#India,#mapwithai",0,1006969338360
4,TZA,JOSM/1.5 (18463 en),2023,{},"#Tanzania,#mapwithai",0,841542487332
...,...,...,...,...,...,...,...
82556,USA,JOSM/1.5 (18700 en),2023,"{highway -> residential, tiger:county -> Orang...","#buildingmapping,#mapwithai",329,13331490
82557,USA,JOSM/1.5 (18700 en),2023,"{highway -> residential, tiger:county -> Orang...","#buildingmapping,#mapwithai",340,13303745
82558,USA,JOSM/1.5 (18700 en),2023,"{highway -> residential, tiger:county -> Orang...","#buildingmapping,#mapwithai",399,13303745
82559,USA,JOSM/1.5 (18565 en),2023,"{highway -> tertiary, tiger:county -> Orange, ...","#buildingmapping,#mapwithai",697,13300336
